# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f309c138f70>
├── 'a' --> tensor([[ 0.7027,  2.2995,  0.6211],
│                   [ 0.7052,  0.1130, -0.7534]])
└── 'x' --> <FastTreeValue 0x7f309c138f40>
    └── 'c' --> tensor([[-0.5012,  0.4784,  0.0666,  0.5823],
                        [-0.2759,  0.4881,  0.1142,  0.6298],
                        [ 0.1480, -0.5355,  1.7297, -0.1985]])

In [4]:
t.a

tensor([[ 0.7027,  2.2995,  0.6211],
        [ 0.7052,  0.1130, -0.7534]])

In [5]:
%timeit t.a

61.3 ns ± 0.0167 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f309c138f70>
├── 'a' --> tensor([[ 0.8709, -2.6227, -0.0210],
│                   [-0.7228, -1.4850,  0.0877]])
└── 'x' --> <FastTreeValue 0x7f309c138f40>
    └── 'c' --> tensor([[-0.5012,  0.4784,  0.0666,  0.5823],
                        [-0.2759,  0.4881,  0.1142,  0.6298],
                        [ 0.1480, -0.5355,  1.7297, -0.1985]])

In [7]:
%timeit t.a = new_value

69.3 ns ± 0.0431 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7027,  2.2995,  0.6211],
               [ 0.7052,  0.1130, -0.7534]]),
    x: Batch(
           c: tensor([[-0.5012,  0.4784,  0.0666,  0.5823],
                      [-0.2759,  0.4881,  0.1142,  0.6298],
                      [ 0.1480, -0.5355,  1.7297, -0.1985]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7027,  2.2995,  0.6211],
        [ 0.7052,  0.1130, -0.7534]])

In [11]:
%timeit b.a

53.9 ns ± 0.0285 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2460, -1.5389, -0.3532],
               [ 0.3847, -0.7262,  1.4160]]),
    x: Batch(
           c: tensor([[-0.5012,  0.4784,  0.0666,  0.5823],
                      [-0.2759,  0.4881,  0.1142,  0.6298],
                      [ 0.1480, -0.5355,  1.7297, -0.1985]]),
       ),
)

In [13]:
%timeit b.a = new_value

477 ns ± 0.0759 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

864 ns ± 0.659 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 347 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 454 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f309c0e0670>
├── 'a' --> tensor([[[ 0.7027,  2.2995,  0.6211],
│                    [ 0.7052,  0.1130, -0.7534]],
│           
│                   [[ 0.7027,  2.2995,  0.6211],
│                    [ 0.7052,  0.1130, -0.7534]],
│           
│                   [[ 0.7027,  2.2995,  0.6211],
│                    [ 0.7052,  0.1130, -0.7534]],
│           
│                   [[ 0.7027,  2.2995,  0.6211],
│                    [ 0.7052,  0.1130, -0.7534]],
│           
│                   [[ 0.7027,  2.2995,  0.6211],
│                    [ 0.7052,  0.1130, -0.7534]],
│           
│                   [[ 0.7027,  2.2995,  0.6211],
│                    [ 0.7052,  0.1130, -0.7534]],
│           
│                   [[ 0.7027,  2.2995,  0.6211],
│                    [ 0.7052,  0.1130, -0.7534]],
│           
│                   [[ 0.7027,  2.2995,  0.6211],
│                    [ 0.7052,  0.1130, -0.7534]]])
└── 'x' --> <FastTreeValue 0x7f309c0e0e80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 62.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2ff0dfa7f0>
├── 'a' --> tensor([[ 0.7027,  2.2995,  0.6211],
│                   [ 0.7052,  0.1130, -0.7534],
│                   [ 0.7027,  2.2995,  0.6211],
│                   [ 0.7052,  0.1130, -0.7534],
│                   [ 0.7027,  2.2995,  0.6211],
│                   [ 0.7052,  0.1130, -0.7534],
│                   [ 0.7027,  2.2995,  0.6211],
│                   [ 0.7052,  0.1130, -0.7534],
│                   [ 0.7027,  2.2995,  0.6211],
│                   [ 0.7052,  0.1130, -0.7534],
│                   [ 0.7027,  2.2995,  0.6211],
│                   [ 0.7052,  0.1130, -0.7534],
│                   [ 0.7027,  2.2995,  0.6211],
│                   [ 0.7052,  0.1130, -0.7534],
│                   [ 0.7027,  2.2995,  0.6211],
│                   [ 0.7052,  0.1130, -0.7534]])
└── 'x' --> <FastTreeValue 0x7f2ff0dfa790>
    └── 'c' --> tensor([[-0.5012,  0.4784,  0.0666,  0.5823],
                        [-0.2759,  0.4881,  0.1142,  0.6298],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 51.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 51 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.5012,  0.4784,  0.0666,  0.5823],
                       [-0.2759,  0.4881,  0.1142,  0.6298],
                       [ 0.1480, -0.5355,  1.7297, -0.1985]],
              
                      [[-0.5012,  0.4784,  0.0666,  0.5823],
                       [-0.2759,  0.4881,  0.1142,  0.6298],
                       [ 0.1480, -0.5355,  1.7297, -0.1985]],
              
                      [[-0.5012,  0.4784,  0.0666,  0.5823],
                       [-0.2759,  0.4881,  0.1142,  0.6298],
                       [ 0.1480, -0.5355,  1.7297, -0.1985]],
              
                      [[-0.5012,  0.4784,  0.0666,  0.5823],
                       [-0.2759,  0.4881,  0.1142,  0.6298],
                       [ 0.1480, -0.5355,  1.7297, -0.1985]],
              
                      [[-0.5012,  0.4784,  0.0666,  0.5823],
                       [-0.2759,  0.4881,  0.1142,  0.6298],
                       [ 0.1480, -0.5355,  1.7297, -0.1985]],

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.5012,  0.4784,  0.0666,  0.5823],
                      [-0.2759,  0.4881,  0.1142,  0.6298],
                      [ 0.1480, -0.5355,  1.7297, -0.1985],
                      [-0.5012,  0.4784,  0.0666,  0.5823],
                      [-0.2759,  0.4881,  0.1142,  0.6298],
                      [ 0.1480, -0.5355,  1.7297, -0.1985],
                      [-0.5012,  0.4784,  0.0666,  0.5823],
                      [-0.2759,  0.4881,  0.1142,  0.6298],
                      [ 0.1480, -0.5355,  1.7297, -0.1985],
                      [-0.5012,  0.4784,  0.0666,  0.5823],
                      [-0.2759,  0.4881,  0.1142,  0.6298],
                      [ 0.1480, -0.5355,  1.7297, -0.1985],
                      [-0.5012,  0.4784,  0.0666,  0.5823],
                      [-0.2759,  0.4881,  0.1142,  0.6298],
                      [ 0.1480, -0.5355,  1.7297, -0.1985],
                      [-0.5012,  0.4784,  0.0666,  0.5823],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 248 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 957 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
